### Diatom Analysis Pipeline - OTU

#### A notebook  for the diatom analysis pipeline using OTU classification.

### 1. Create sub-selection of samples for local use

#### All 2016 samples that meet selection criteria

In [1]:
import pandas as pd
df = pd.read_csv('2016 Data.csv')
df.head()

,FERAID,NGS Run Number,BIOSYS site ID,Season,Dataset,SPT_Name,Conductivity,N_NH4,N_NO3,P_PO4,Alkalinity,pH,TotalRead,NonPlankRead,NoBlast Hit,% No Blast Hit of total count,NGS TDI,Northing,Easting
0,2662,Diatom2,53,Spr,2016,MASHAM,175.611222,0.030000,1.022745,0.020203,50.416667,8.047222,30710,30706,12770,41.587963,40.267399,NaN,NaN
1,2854,Diatom4,53,Aut,2016,MASHAM,175.611222,0.030000,1.022745,0.020203,50.416667,8.047222,49744,49696,18357,36.938587,59.602807,NaN,NaN
2,2632,Diatom2,83,Spr,2016,COLNEBRIDGE,323.866253,0.109224,2.365707,0.091403,44.250000,7.882188,22553,21006,1771,8.430924,48.468433,NaN,NaN
3,2700,Diatom3,83,Aut,2016,COLNEBRIDGE,323.866253,0.109224,2.365707,0.091403,44.250000,7.882188,57772,45069,12879,28.576183,71.478087,NaN,NaN
4,2987,Diatom1,110,Spr,2016,HARTLINGTON BRIDGE,113.406089,0.015710,0.523256,0.014135,37.500000,7.857308,27403,27403,15705,57.311243,39.948032,NaN,NaN


#### Sample across the TDI range 
#### (Actually this is wrong, as we really want to create n samples of sites (each with 4 sequence files)

#### We want to have representative set of samples across TDI range. Group continuous TDI values into 10 bins.

In [18]:
binned = pd.cut(df['NGS TDI'], 10)
df['Binned'] = pd.cut(df['NGS TDI'], 10)
grouped = df.groupby(df['Binned'])

#### Select n samples from each bin
##### (n * 10 * 2 sequence files)

In [51]:
sub_selection = grouped.apply(lambda x: x.sample(n=2, replace=True))

#### Copy selected forward and reverse read files into working directory

In [75]:
from shutil import copyfile
for index, row in sub_selection.iterrows():
    for n in range(1,3):
        from_file = "sequences/complete_set/{}.R{}.fastq.gz".format(row['FERAID'],n)
        to_file = "sequences/{}.R{}.fastq.gz".format(row['FERAID'],n)
        print(from_file, to_file)
        copyfile(from_file, to_file)    

('sequences/complete_set/3096.R1.fastq.gz', 'sequences/3096.R1.fastq.gz')
('sequences/complete_set/3096.R2.fastq.gz', 'sequences/3096.R2.fastq.gz')
('sequences/complete_set/2988.R1.fastq.gz', 'sequences/2988.R1.fastq.gz')
('sequences/complete_set/2988.R2.fastq.gz', 'sequences/2988.R2.fastq.gz')
('sequences/complete_set/3088.R1.fastq.gz', 'sequences/3088.R1.fastq.gz')
('sequences/complete_set/3088.R2.fastq.gz', 'sequences/3088.R2.fastq.gz')
('sequences/complete_set/3059.R1.fastq.gz', 'sequences/3059.R1.fastq.gz')
('sequences/complete_set/3059.R2.fastq.gz', 'sequences/3059.R2.fastq.gz')
('sequences/complete_set/2947.R1.fastq.gz', 'sequences/2947.R1.fastq.gz')
('sequences/complete_set/2947.R2.fastq.gz', 'sequences/2947.R2.fastq.gz')
('sequences/complete_set/2866.R1.fastq.gz', 'sequences/2866.R1.fastq.gz')
('sequences/complete_set/2866.R2.fastq.gz', 'sequences/2866.R2.fastq.gz')
('sequences/complete_set/3141.R1.fastq.gz', 'sequences/3141.R1.fastq.gz')
('sequences/complete_set/3141.R2.fastq

### 2. Create BLAST reference database

In [78]:
%%time
!makeblastdb -in diatoms.sequences.FINAL2017.fasta -out diatoms -dbtype nucl



Building a new DB, current time: 06/07/2019 13:53:23
New DB name:   /code/diatoms
New DB title:  diatoms.sequences.FINAL2017.fasta
Sequence type: Nucleotide
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 2701 sequences in 0.1822 seconds.
CPU times: user 0 ns, sys: 20 ms, total: 20 ms
Wall time: 1.03 s


### 3. Quality Control
* *Cutadapt:* Trim primers from sequences
* *SicklePE:* Trim off bad quality 3' bases
* *Pear:* Merge R1 and R2 reads
* *SickleSE:* Remove post-merging bad quality sequences
* *Histogram Generation:* Plots sequence length against number of sequences
* *QIIME Prep:* Pepare passed QC files for QIIME processing

In [79]:
%%time
!python ./ampliconQC.py --data sequences --forward ATGCGTTGGAGAGARCGTTTC --reverse GATCACCTTCTAATTTACCWACAACTG --threads 8 --histograms --qiime

This is cutadapt 1.9.1 with Python 2.7.16
Command line parameters: -e 0.047619047619 -b ATGCGTTGGAGAGARCGTTTC -b GATCACCTTCTAATTTACCWACAACTG -o /code/sequences/2385.R2.fastq.gz.trimmed.fastq.gz /code/sequences/2385.R2.fastq.gz
Trimming 2 adapters with at most 4.8% errors in single-end mode ...
Finished in 1.05 s (23 us/read; 2.65 M reads/minute).

=== Summary ===

Total reads processed:                  46,371
Reads with adapters:                    40,033 (86.3%)
Reads written (passing filters):        46,371 (100.0%)

Total basepairs processed:    12,647,633 bp
Total written (filtered):     11,568,466 bp (91.5%)

=== Adapter 1 ===

Sequence: ATGCGTTGGAGAGARCGTTTC; Type: variable 5'/3'; Length: 21; Trimmed: 55 times.
6 times, it overlapped the 5' end of a read
49 times, it overlapped the 3' end or was within the read

No. of allowed errors:
0-21 bp: 0

Overview of removed sequences (5')
length	count	expect	max.err	error counts
3	3	724.5	0	3
4	3	181.1	0	3


Overview of removed sequence


FastQ paired records kept: 179706 (89853 pairs)
FastQ single records kept: 4472 (from PE1: 3896, from PE2: 576)
FastQ paired records discarded: 804 (402 pairs)
FastQ single records discarded: 4472 (from PE1: 576, from PE2: 3896)



FastQ paired records kept: 151822 (75911 pairs)
FastQ single records kept: 5137 (from PE1: 3780, from PE2: 1357)
FastQ paired records discarded: 2106 (1053 pairs)
FastQ single records discarded: 5137 (from PE1: 1357, from PE2: 3780)



FastQ paired records kept: 232102 (116051 pairs)
FastQ single records kept: 3036 (from PE1: 2495, from PE2: 541)
FastQ paired records discarded: 456 (228 pairs)
FastQ single records discarded: 3036 (from PE1: 541, from PE2: 2495)



FastQ paired records kept: 243418 (121709 pairs)
FastQ single records kept: 3339 (from PE1: 2898, from PE2: 441)
FastQ paired records discarded: 426 (213 pairs)
FastQ single records discarded: 3339 (from PE1: 441, from PE2: 2898)



FastQ paired records kept: 104572 (52286 pairs)
FastQ single reco

 ____  _____    _    ____ 
|  _ \| ____|  / \  |  _ \
| |_) |  _|   / _ \ | |_) |
|  __/| |___ / ___ \|  _ <
|_|   |_____/_/   \_\_| \_\

PEAR v0.9.11 [Nov 5, 2017]

Citation - PEAR: a fast and accurate Illumina Paired-End reAd mergeR
Zhang et al (2014) Bioinformatics 30(5): 614-620 | doi:10.1093/bioinformatics/btt593

Forward reads file.................: /code/sequences/3141.sickle.trimmed.R1.fastq.gz
Reverse reads file.................: /code/sequences/3141.sickle.trimmed.R2.fastq.gz
PHRED..............................: 33
Using empirical frequencies........: NO
Statistical method.................: OES
Maximum assembly length............: 999999
Minimum assembly length............: 50
p-value............................: 0.010000
Quality score threshold (trimming).: 0
Minimum read size after trimming...: 1
Maximal ratio of uncalled bases....: 1.000000
Minimum overlap....................: 10
Scoring method.....................: Scaled score
Threads............................: 8

Allo

 ____  _____    _    ____ 
|  _ \| ____|  / \  |  _ \
| |_) |  _|   / _ \ | |_) |
|  __/| |___ / ___ \|  _ <
|_|   |_____/_/   \_\_| \_\

PEAR v0.9.11 [Nov 5, 2017]

Citation - PEAR: a fast and accurate Illumina Paired-End reAd mergeR
Zhang et al (2014) Bioinformatics 30(5): 614-620 | doi:10.1093/bioinformatics/btt593

Forward reads file.................: /code/sequences/2532.sickle.trimmed.R1.fastq.gz
Reverse reads file.................: /code/sequences/2532.sickle.trimmed.R2.fastq.gz
PHRED..............................: 33
Using empirical frequencies........: NO
Statistical method.................: OES
Maximum assembly length............: 999999
Minimum assembly length............: 50
p-value............................: 0.010000
Quality score threshold (trimming).: 0
Minimum read size after trimming...: 1
Maximal ratio of uncalled bases....: 1.000000
Minimum overlap....................: 10
Scoring method.....................: Scaled score
Threads............................: 8

Allo

 ____  _____    _    ____ 
|  _ \| ____|  / \  |  _ \
| |_) |  _|   / _ \ | |_) |
|  __/| |___ / ___ \|  _ <
|_|   |_____/_/   \_\_| \_\

PEAR v0.9.11 [Nov 5, 2017]

Citation - PEAR: a fast and accurate Illumina Paired-End reAd mergeR
Zhang et al (2014) Bioinformatics 30(5): 614-620 | doi:10.1093/bioinformatics/btt593

Forward reads file.................: /code/sequences/2947.sickle.trimmed.R1.fastq.gz
Reverse reads file.................: /code/sequences/2947.sickle.trimmed.R2.fastq.gz
PHRED..............................: 33
Using empirical frequencies........: NO
Statistical method.................: OES
Maximum assembly length............: 999999
Minimum assembly length............: 50
p-value............................: 0.010000
Quality score threshold (trimming).: 0
Minimum read size after trimming...: 1
Maximal ratio of uncalled bases....: 1.000000
Minimum overlap....................: 10
Scoring method.....................: Scaled score
Threads............................: 8

Allo

Histogram /code/sequences/readyForQiime.allsamples.fasta.histogram.svg finished
Histogram /code/sequences/3141.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/3059.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/2966.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/2973.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/2385.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/2532.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/2988.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/3143.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/3208.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/2866.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/3096.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/2823.passedQC.fastq.histogram.svg finished
Histogram /code/sequences/2753.passedQC.fastq.histogram.svg finished
Histogram /code/sequenc

### 4. Generate sequence counts file used in final step to produce reports

In [80]:
%%time
!for file in sequences/*.passedQC.fastq; \
do \
  awk 'NR%4==2{sum+=1}END{print FILENAME,sum}' $file >> sequences/diatomSequenceCounts.txt; \
done

CPU times: user 40 ms, sys: 20 ms, total: 60 ms
Wall time: 2.96 s


### 5. Assign similar sequences to OTUs using user-defined similarity threshold
#### Default clustering algorithm is UCLUST, O(m\*n), which is a greedy algorithm and is dependent on ordering of sequences in readyForQiime.allsamples.fasta file.

In [81]:
%%time
!pick_otus.py -i sequences/readyForQiime.allsamples.fasta -o sequences/picked_otus_97

/miniconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
CPU times: user 8.83 s, sys: 1.78 s, total: 10.6 s
Wall time: 10min 44s


###  6. Pick a representative set of sequences. For each OTU, one sequence will be used in subsequent analysis

In [82]:
%%time
!pick_rep_set.py -i sequences/picked_otus_97/readyForQiime.allsamples_otus.txt \
  -f sequences/readyForQiime.allsamples.fasta \
  -o sequences/repset.fasta

CPU times: user 360 ms, sys: 100 ms, total: 460 ms
Wall time: 27 s


### 7. Query BLAST database with OTU representatives

In [83]:
%%time
!blastn -db diatoms -query sequences/repset.fasta \
  -out sequences/repset.diatoms.blastn \
  -task blastn -max_target_seqs 1 -num_threads 8 -outfmt 6 -evalue 0.01

CPU times: user 58.5 s, sys: 10.3 s, total: 1min 8s
Wall time: 1h 12min 34s


In [84]:
!mkdir sequences/assigned_taxonomy

### 8. Create taxonomy assignments from BLAST outputs

In [85]:
%%time
!python ./create_taxonomy_assignments_from_blast.py --taxonomy diatoms.taxonomy.FINAL2017.txt \
  --percid 95.0 --blast sequences/repset.diatoms.blastn --output sequences/assigned_taxonomy/repset.taxonomy.txt 

CPU times: user 0 ns, sys: 30 ms, total: 30 ms
Wall time: 1.89 s


### 9. Report how often an OTU is found in each sample and add the taxonomic predictions for each OTU

In [86]:
%%time
!make_otu_table.py -i sequences/picked_otus_97/readyForQiime.allsamples_otus.txt \
  -t sequences/assigned_taxonomy/repset.taxonomy.txt \
  -o sequences/otu_table.biom

CPU times: user 210 ms, sys: 20 ms, total: 230 ms
Wall time: 14.2 s


### 10. Filter the OTU table based on taxonomic metadata excluding specific taxa

In [87]:
%%time
!filter_taxa_from_otu_table.py -i sequences/otu_table.biom \
  -o sequences/otu_table.diatomsonly.biom \
  -n MARINE,NOT_DIATOM,Yellow_green_Algae,None

CPU times: user 190 ms, sys: 70 ms, total: 260 ms
Wall time: 16.5 s


### 11. Sort OTU table by sample id

In [88]:
%%time
!sort_otu_table.py -i sequences/otu_table.diatomsonly.biom \
  -o sequences/otu_table.diatomsonly.biom

CPU times: user 220 ms, sys: 50 ms, total: 270 ms
Wall time: 15.7 s


### 12. Provide summary information of the representation of taxonomic groups within each sample

In [89]:
%%time
!summarize_taxa.py -L 1 \
  -i sequences/otu_table.diatomsonly.biom \
  -o sequences/visualised_taxonomy -a

CPU times: user 300 ms, sys: 50 ms, total: 350 ms
Wall time: 21.3 s


### 13. Produce Diatom reports

In [90]:
%%time
!python ./produceDiatomReports.py --folder sequences --lookup lookuptable.txt

Reports completed
CPU times: user 0 ns, sys: 20 ms, total: 20 ms
Wall time: 1.41 s


### 13. Inspect producted Diatom reports

In [91]:
import pandas as pd

In [92]:
pd.read_csv('sequences/Abundances.fail.csv')

,Taxonomy,101522615,101608272,101247590,100966229,100775150,101481781,101493216,101611476,101607102,...,101712111,101365185,100985450,101537286,101712096,100757042,101712098,101560493,101280443,101581652


In [93]:
pd.read_csv('sequences/Abundances.pass.csv')

,Taxonomy,100966229,101712111,101280443,101712098,101607102,101560493,101712096,100775150,101581652,...,101306260,101481781,100985450,101522615,101493216,101537286,101247590,101365185,100757042,101611476
0,AC023A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,18.0,2.0,0.0,0.0,0.0,12.0,0.0
1,AC143A,0.0,827.0,1.0,3799.0,0.0,7.0,5336.0,166.0,7.0,...,771.0,1.0,1.0,0.0,0.0,606.0,0.0,3.0,0.0,38.0
2,AC166A,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ACHD-02,844.0,22.0,1352.0,282.0,1552.0,701.0,52.0,42.0,23.0,...,127.0,335.0,674.0,216.0,19.0,124.0,4.0,58.0,5.0,16.0
4,ACHD-09,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,ACHD-11,1.0,2.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,ACHD-12,434.0,82.0,7.0,74.0,0.0,195.0,0.0,76.0,0.0,...,0.0,0.0,17.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
7,ADLA-01,5.0,0.0,1.0,0.0,0.0,2.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
8,ADLA-03,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,ADLA-05,0.0,0.0,1.0,0.0,0.0,0.0,14.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
